Hello everyone!

Today we're to make Python act like C++.

By a raise of hands - who here uses C++?
Uses C++ as their main language?

Uses Python?
Python as a main language?

Who prefers Python?
Who prefers C++?

Anyhow, C++ is a large language. Very large.

So today we'll focus on my favourite C++ feature.

Destructors!

I started seriously learning & working in C++ after several years
of development in C and Python.

Both when coming from a lower-level language like C,
and coming from a supposedly higher-level language like Python,
destructors are an exciting new addition to a programmer's toolbox.

Destructors have 3 main properties we're going to discuss today.
They are:

1. Automatic
2. Composable
3. Implicit

## Automatic

Dtors are called automatically when you leave the scope.
It doesn't matter if you return, or raise an exception.
Doesn't matter if you have 1 return statement in your
function or 10, the destructors are automatically called where needed.

```C++
// Without Dtors

auto read_file(string path) -> string {
    FileReader file{path};
    string content = file.read();
    file.close();
    return content;
}
```

```C++
// Without Dtors, with exceptions

auto read_file(string path) -> string {
    FileReader file{path};
    try {
        string content = file.read();
        return content;
    } finally {
        file.close();
    }
}
```

```C++
// With Dtors, with exceptions

auto read_file(string path) -> string {
    FileReader file{path};
    string content = file.read();
    return content;
}
```

## Composable

Our objects can have members with dtors.
In those cases, the dtors of the members are automatically
called in the right time as well.
We don't need to write any extra code to make it work.

## Implicit

Destructors do not affect the usage of our objects.
They are enabled by default (though we _can_ make them private...)

Here's a code example:

```C++

// Without dtors

MyType instance{};

// With dtors

MyType instance{};

```
The code is exactly the same.

This is a key part.
It gives us the freedom to add dtors when we need to.
It also gives us the ability to freely compose our types
with no concerns about the interfaces.

Now let's have a look at Python.

First,

## Automatic

In Python, we don't have destructors, but we have context managers.
It is also very common to use exceptions.

In [ ]:
def read_file(path: str) -> str:
    file = FileReader(path)
    try:
        return file.read()
    finally:
        file.close()


def read_file(path: str) -> str:
    file = FileReader(path)
    with file:
        return file.read()

As you can see - instead of creating a variable, we're using
a `with` statement.
The `with` statement creats a new indented block.
When entering the block, it calls `file`'s `__enter__` method.
When leaving the block, via regular flow or due to
an exception - it'll call `file`'s `__exit__` method.
If an exception was raised, the `__exit__` method has the chance
to handle it or re-throw it.

Here, we close the file without handling the exception.

In [ ]:
class FileReader:
    ...

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.close()

So we have Automatic down, for the most part.

## Composable

Python offers no such thing.
Sure, we can store member variables,
but there's no mechanism to automatically call their `__exit__` methods.
We have to do it ourselves.

In [ ]:
class FileHolder:
    ...

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.file1.close()
        self.file2.close()

## Implicit

As with already seen - Python's scope-based destruction is
very explicit.
We have to write our code with it in mind.

But... Why does it matter so much?

Let's look at a code example.

This is a simplification of some production code I recently had to change.

In [ ]:
class ArchiveReader:
    def __init__(self, path: str):
        self.data = {}
        with ZipFile(path) as zipfile:
            for name in zipfile.namelist():
                with zipfile.open(name) as f:
                    self.data[name] = f.read()

    def read(self, name):
        return self.data[name]

We have some data stored in a multi-file zip archive.
Since the zipfiles were originally very small, we just loaded
all the data into memory, and returned the relevant data
when requested.

Later, however, the files have gotten larger.
Too large to keep in memory.
So now, we have to open the zipfile when we create our object,
and close it when we're destructing our object.

In [ ]:
class BigArchiveReader:
    def __init__(self, path: str):
        self.zipfile = ZipFile(path)

    def read(self, name: str):
        with self.zipfile.open(name) as f:
            return parse(f.read())

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.zipfile.close()
        return False

As Python does not have any built-in composability for context-managers,
we need to convert our class into a context-manager as well.

Additionally, as context-managers are not implicit, this would affect the
usage of our class:

In [ ]:
reader = ArchiveReader("corecpp.zip")
print(reader.read("2021"))

with BigArchiveReader("corecpp.zip") as big_reader:
    print(reader.read("2021"))

If used in one place, this is not a significant change.
However, if the class is commonly used, or is used in code we cannot
change, we won't be able to change the behaviour of our class.

If possible, we want to inch closer to what C++ has to offer.
To have our code be something along the lines of

In [ ]:
class BestArchiveReader:
    zipfile: ZipFile
    def __init__(self, path: str):
        self.zipfile = ZipFile(path)

    def read(self, name: str):
        with self.zipfile.open(name) as f:
            return f.read()

In [ ]:
reader = BestArchiveReader("corecpp.zip")
print(reader.read("2021"))

But with the semantics of the `BigArchiveReader`.
Having the zipfile automatically closed when `BestArchiveReader`
goes out of scope.

Now that we know what we wanna do - it's time to head into the implementation.

Just note that due to implementation details, we'll handle `implicit`
before we handle `composable`.